In [14]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [15]:
#Read csv
file_path = 'Cleaned_data_with_population.csv'

df = pd.read_csv(file_path)

# Sort by LOCATION and TIME for correct growth calculation
df.sort_values(['LOCATION', 'TIME'], inplace=True)
# Calculate growth rates for selected columns
growth_columns = ['PC_HEALTHXP', 'PC_GDP', 'USD_CAP','TOTAL_SPEND','POPULATION']
for col in growth_columns:
    df[f'{col}_GROWTH'] = df.groupby('LOCATION')[col].pct_change()

# Identify the column with the highest growth each year for each location
df['HIGHEST_GROWTH_METRIC'] = df[[f'{col}_GROWTH' for col in growth_columns]].idxmax(axis=1)
df['HIGHEST_GROWTH_VALUE'] = df[[f'{col}_GROWTH' for col in growth_columns]].max(axis=1)

# Optional: clean the metric name (remove "_GROWTH" suffix)
df['HIGHEST_GROWTH_METRIC'] = df['HIGHEST_GROWTH_METRIC'].str.replace('_GROWTH', '')

# Save the result to csv
df.to_csv('Cleaned_data_with_population_growth.csv', index=False)




C:\Users\bmanna\AppData\Local\Temp\ipykernel_5872\2481617357.py:14: FutureWarning:

The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError



In [ ]:
# Calculate average growth and total spend per location
summary = df.groupby('LOCATION').agg({
    'USD_CAP_GROWTH': 'mean',
    'TOTAL_SPEND': 'mean'
}).reset_index()

# Rename for clarity
summary.rename(columns={
    'USD_CAP_GROWTH': 'AVG_USD_CAP_GROWTH',
    'TOTAL_SPEND': 'AVG_TOTAL_SPEND'
}, inplace=True)

# Determine thresholds (top 25% for growth, bottom 25% for spend)
growth_threshold = summary['AVG_USD_CAP_GROWTH'].quantile(0.75)
spend_threshold = summary['AVG_TOTAL_SPEND'].quantile(0.25)

# Identify countries of interest
summary["TARGET"] = summary.apply(
    lambda row: "Potential" if (row["AVG_USD_CAP_GROWTH"] >= growth_threshold and row["AVG_TOTAL_SPEND"] <= spend_threshold)
    else "Other",
    axis=1
)

# Interactive Plotly scatter plot
fig = px.scatter(
    summary,
    x="AVG_TOTAL_SPEND",
    y="AVG_USD_CAP_GROWTH",
    color="TARGET",
    text="LOCATION",
    hover_data=["LOCATION", "AVG_USD_CAP_GROWTH", "AVG_TOTAL_SPEND"],
    color_discrete_map={"Potential": "red", "Other": "#20557b"},
    labels={
        "AVG_TOTAL_SPEND": "Avg Total Spend",
        "AVG_USD_CAP_GROWTH": "Avg USD_CAP Growth Rate",
        "TARGET": "Country Type"
    },
    title="Countries with High USD_CAP Growth and Low Total Spend"
)

# Improve layout
fig.update_traces(textposition='top center')
fig.update_layout(legend=dict(title="Target Country"))
fig.show()

# Create interactive Choropleth map
fig = px.choropleth(
    summary,
    locations="LOCATION",
    color="TARGET",
    locationmode="ISO-3",
    hover_name="LOCATION",
    hover_data={"AVG_USD_CAP_GROWTH": True, "AVG_TOTAL_SPEND": True},
    color_discrete_map={"Potential": "green", "Other": "#ef98da"},
    title="Countries with High USD_CAP Growth and Low Total Spend"
)

fig.update_layout(geo=dict(showframe=False, showcoastlines=True))
fig.show()



In [36]:
# Calculate average growth and total spend per location
summary = df.groupby('LOCATION').agg({
    'USD_CAP_GROWTH': 'mean',
    'TOTAL_SPEND': 'mean'
}).reset_index()

# Rename for clarity
summary.rename(columns={
    'USD_CAP_GROWTH': 'AVG_USD_CAP_GROWTH',
    'TOTAL_SPEND': 'AVG_TOTAL_SPEND'
}, inplace=True)

# Create 3 quantile bins (33.3%, 66.7%, 100%)
summary["USD_CAP_GROWTH_QUANTILE"] = pd.qcut(
    summary["AVG_USD_CAP_GROWTH"],
    q=3,
    labels=["Low", "Medium", "High"]
)

summary["AVG_TOTAL_SPEND_QUANTILE"] = pd.qcut(
    summary["AVG_TOTAL_SPEND"],
    q=3,
    labels=["Low", "Medium", "High"]
)

# Determine thresholds (top 25% for growth, bottom 25% for spend)

# Identify countries of interest
summary["TARGET"] = summary.apply(
    lambda row: "High Potential" if (row["USD_CAP_GROWTH_QUANTILE"] == 'High' and row["AVG_TOTAL_SPEND_QUANTILE"] == 'Low')
    else "Medium Potential" if (
        (row["USD_CAP_GROWTH_QUANTILE"] == 'High' and row["AVG_TOTAL_SPEND_QUANTILE"] == 'Medium') or
        (row["USD_CAP_GROWTH_QUANTILE"] == 'Medium' and row["AVG_TOTAL_SPEND_QUANTILE"] == 'Low')
    )
    else "Low Potential",
    axis=1
)


# Create interactive Choropleth map
fig = px.choropleth(
    summary,
    locations="LOCATION",
    color="TARGET",
    locationmode="ISO-3",
    hover_name="LOCATION",
    hover_data={"AVG_USD_CAP_GROWTH": True, "AVG_TOTAL_SPEND": True},
    color_discrete_map={"High Potential": "green", "Low Potential": "#ef98da", "Medium Potential" : "#efe298"},
    title="Countries with High USD_CAP Growth and Low Total Spend"
)

fig.update_layout(geo=dict(showframe=False, showcoastlines=True))
fig.show()
